In [1]:
import gc
import time
from logzero import logger
import numpy as np
import pandas as pd

In [2]:
ls -lah ../data/

total 24G
drwxrwxr-x 5 ubuntu ubuntu 4.0K May  5 12:36 ./
drwxrwxr-x 8 ubuntu ubuntu 4.0K Apr 30 17:30 ../
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 16:52 .ipynb_checkpoints/
-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 13:04 count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 count_user_by_channel_app.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 29 14:21 kenkoooos/
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 merge.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 17:13 raw/
-rw-rw-r-- 1 ubuntu ubuntu 1.4G Apr 28 17:20 raw.zip
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May

In [ ]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=8)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.9 µs


In [ ]:
merge = merge[merge.is_test == False]
gc.collect()

In [ ]:
print(merge.shape)
merge.dtypes

### preparation

In [6]:
%%time
channel_grp = merge.groupby('app', as_index=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 246 µs


In [7]:
del merge
gc.collect()

0

### simple count

In [8]:
%%time
count_by_app = (channel_grp.ip.count().rename(columns={'ip': 'count_by_app'}))

CPU times: user 2.37 s, sys: 1.21 s, total: 3.58 s
Wall time: 3.58 s


In [9]:
print(count_by_app.shape)
count_by_app.head()

(674, 2)


,app,count_by_app
0,0,2609
1,1,3795936
2,2,14878211
3,3,23614633
4,4,89155


In [10]:
%%time
attrib_by_app = (channel_grp.is_attributed.sum().rename(columns={'is_attributed': 'attrib_by_app'})).astype('uint32')

CPU times: user 1.01 s, sys: 584 ms, total: 1.59 s
Wall time: 1.59 s


In [11]:
attrib_by_app.head()

,app,attrib_by_app
0,0,791
1,1,617
2,2,3504
3,3,6900
4,4,3


In [12]:
agg_by_app = pd.concat([count_by_app, 
                        attrib_by_app['attrib_by_app']], axis=1)

In [13]:
agg_by_app.to_feather('../data/agg_by_app.feather')

In [14]:
del agg_by_app
gc.collect()

14

In [15]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 12:36 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 12:31 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### unique count

In [16]:
%%time
uqcount_by_app = pd.concat([
    channel_grp.channel.nunique().to_frame('uqcount_channel_by_app')
    , channel_grp.ip.nunique().to_frame('uqcount_ip_by_app')
    , channel_grp.device.nunique().to_frame('uqcount_device_by_app')
    , channel_grp.os.nunique().to_frame('uqcount_os_by_app')
    , channel_grp.ip_os_device_dow_hour.nunique().to_frame('uqcount_user_by_app')
], axis=1)

CPU times: user 3min 28s, sys: 6.09 s, total: 3min 34s
Wall time: 3min 39s


In [17]:
uqcount_by_app.head()

,uqcount_channel_by_app,uqcount_ip_by_app,uqcount_device_by_app,uqcount_os_by_app,uqcount_user_by_app
0,1,1139,227,14,1417
1,32,113943,6,209,2479017
2,27,135317,6,351,5288033
3,47,155393,7,280,7219771
4,1,10502,6,104,38886


In [18]:
uqcount_by_app.to_feather('../data/uqcount_by_app.feather')

In [19]:
del uqcount_by_app
gc.collect()

69

In [20]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 12:36 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### cross tabulation(channel)

In [3]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=8)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs


In [4]:
merge = merge[['channel', 'app', 'ip_os_device_dow_hour']][merge.is_test == False]
merge['counter'] = 1
gc.collect()

0

In [5]:
print(merge.shape)
merge.head()

(126923238, 4)


,channel,app,ip_os_device_dow_hour,counter
0,153,23,765080350001103,1
1,134,18,765080350001103,1
2,315,15,765080350001103,1
3,469,11,765080350001103,1
4,379,3,765080350001103,1


In [24]:
merge.drop_duplicates(inplace = True)

In [25]:
print(merge.shape)
merge.head()

(83747805, 4)


,channel,app,ip_os_device_dow_hour,counter
0,153,23,765080350001103,1
1,134,18,765080350001103,1
2,315,15,765080350001103,1
3,469,11,765080350001103,1
4,379,3,765080350001103,1


In [26]:
%%time
count_user_by_channel_app = merge.pivot_table(values  = 'counter',
                                              index   = 'app',
                                              columns = 'channel', 
                                              aggfunc = sum,
                                              fill_value = 0)

CPU times: user 4.62 s, sys: 2.46 s, total: 7.08 s
Wall time: 7.08 s


In [27]:
chlabel = ['ch_{0:03d}'.format(i) for i in list(set(merge.channel))]
count_user_by_channel_app.columns = chlabel
count_user_by_channel_app.reset_index(inplace=True)

In [28]:
count_user_by_channel_app.head()

,app,ch_000,ch_003,ch_004,ch_005,ch_013,ch_014,ch_015,ch_017,ch_018,...,ch_484,ch_486,ch_487,ch_488,ch_489,ch_490,ch_496,ch_497,ch_498,ch_500
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,4,0,0,36209,0,0,147786,5141,...,0,0,0,0,0,0,0,1,0,0
2,2,0,0,0,0,0,0,0,0,26665,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,3942,745567,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
count_user_by_channel_app.tail()

,app,ch_000,ch_003,ch_004,ch_005,ch_013,ch_014,ch_015,ch_017,ch_018,...,ch_484,ch_486,ch_487,ch_488,ch_489,ch_490,ch_496,ch_497,ch_498,ch_500
669,764,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
670,765,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
671,766,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
672,767,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
673,768,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
count_user_by_channel_app.to_feather('../data/count_user_by_app_channel.feather')

In [31]:
del count_user_by_channel_app
gc.collect()

1829

In [32]:
ls -lah ../data/*.feather

-rw-r--r-- 1 ubuntu ubuntu  14K May  5 12:59 ../data/agg_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 4.3K May  5 04:22 ../data/agg_by_channel.feather
-rw-r--r-- 1 ubuntu ubuntu  35K May  5 05:47 ../data/channel_pca.feather
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 ../data/click_data.feather
-rw-r--r-- 1 ubuntu ubuntu  93K May  5 04:53 ../data/count_channel_coincidence_per_user.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 13:04 ../data/count_user_by_app_channel.feather
-rw-r--r-- 1 ubuntu ubuntu 1.1M May  5 04:27 ../data/count_user_by_channel_app.feather
-rw-r--r-- 1 ubuntu ubuntu  18G May  4 07:40 ../data/merge.feather
-rw-r--r-- 1 ubuntu ubuntu  27K May  5 13:03 ../data/uqcount_by_app.feather
-rw-r--r-- 1 ubuntu ubuntu 8.4K May  5 04:26 ../data/uqcount_by_channel.feather


### cross tabulation (apps clicked by the same user)

In [6]:
merge.drop(['channel'], axis=1,  inplace=True)
merge.drop_duplicates(inplace=True)

In [7]:
print(merge.shape)
merge.head()

(57046774, 3)


,app,ip_os_device_dow_hour,counter
0,23,765080350001103,1
1,18,765080350001103,1
2,15,765080350001103,1
3,11,765080350001103,1
4,3,765080350001103,1


In [8]:
gc.collect()

7

In [ ]:
merge['counter'] = merge['counter'].astype('bool')

In [ ]:
d = merge.pivot('ip_os_device_dow_hour', 'app')

In [ ]:
d.fillna(False, inplace=True)

In [ ]:
d.head()

In [ ]:
d.max().max()

In [ ]:
%%time
mat = np.dot(d.astype('bool').astype('uint16').values.T,
             d.astype('bool').astype('uint16').values)

In [ ]:
count_app_coincidence_per_user = pd.DataFrame(mat)

In [ ]:
del mat, d
gc.collect()

In [ ]:
applabel = ['chan_{0:03d}'.format(i) for i in list(set(merge.app))]
count_app_coincidence_per_user.columns = applabel
count_app_coincidence_per_user.head()

In [ ]:
count_app_coincidence_per_user.to_feather('../data/count_channel_coincidence_per_user.feather')

In [ ]:
del count_app_coincidence_per_user
gc.collect()

In [ ]:
ls -lah ../data/*.feather

### pca

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
d = pd.concat([
    pd.read_feather('../data/agg_by_app.feather'),
    pd.read_feather('../data/uqcount_by_app.feather'),
    pd.read_feather('../data/count_user_by_app_channel.feather'),
    pd.read_feather('../data/count_app_coincidence_per_user.feather')
], axis = 1)

In [ ]:
print(d.shape)
d.head()

In [ ]:
y = d[['app', 'attrib_by_channel']].iloc[:, 1:]
X = d.drop(['app', 'attrib_by_channel'], axis=1)

In [ ]:
n_components = 20
pca = PCA(n_components = n_components)
res = pca.fit_transform(X.values)

In [ ]:
res.shape

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
col = ['channel_pca_{0:02d}'.format(i) for i in [i + 1 for i in range(20)]]
res = pd.DataFrame(res, columns=col)
app_pca = pd.concat([y.app, res], axis=1)

In [ ]:
app_pca.to_feather('../data/app_pca.feather')

In [ ]:
app_pca.head()

### とりあえず回帰してみる

In [ ]:
import statsmodels as sm

In [ ]:
X1 = sm.add_constant(
    app_pca.iloc[:, 1:] / app_pca.iloc[:, 1:].max().max()
    , prepend=False
)

In [ ]:
X1.head()

In [ ]:
model = sm.GLM(y.attrib_by_app.values,
               X1.values,
               family=sm.families.Poisson(),
               offset = np.log(X.count_by_app.values)
              )

In [ ]:
print(y.attrib_by_app.shape)
print(X1.shape)
print(X.count_by_app.shape)

In [ ]:
print(model.fit().summary())